In [11]:

import fitz
import camelot
import warnings , math, collections
import pickle
from collections import Counter
import re

warnings.filterwarnings("ignore", category=UserWarning) 

#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"


pickle_path_text = r"\output\pkl\textual_data.pkl"
pickle_path_tab = r"\output\pkl\tabular_data.pkl"
samco_path = r"\files\SamcoFactSheet2024.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"
dry_run_path = r"\output\DryRun.pdf"

In [ ]:
"""Basic page cleaning, image removal, page number removal,
repetitive data removal"""

In [2]:
with open(path + pickle_path_text , 'rb') as file:
    textual_data = pickle.load(file)

In [3]:
#aggregate code
from collections import Counter

def get_pdf_aggregates(block_data):
    pdf_sizes = []
    pdf_colors = []
    pdf_words = []
    
    document = block_data['blocks']
    for page in document:
        page_size, page_color, page_text = [], [], []
        for block in page:
            for line in block['lines']:
                for span in line['spans']:
                    page_color.append(span['color'])
                    page_size.append(span['size'])
                    page_text.append(span['text'])
        pdf_colors.append(page_color)
        pdf_sizes.append(page_size)
        pdf_words.append(page_text)

    grand_pdf_colors = Counter([color for sublist in pdf_colors for color in sublist])
    grand_pdf_sizes = Counter([size for sublist in pdf_sizes for size in sublist])
    grand_pdf_words = Counter([word for sublist in pdf_words for word in sublist])

    return {
        "grand": {
            "text": grand_pdf_words,
            "color": grand_pdf_colors,
            "size": grand_pdf_sizes
        },
        "page": {
            'text': [Counter(page) for page in pdf_words],
            'size': [Counter(page) for page in pdf_sizes],
            'color': [Counter(page) for page in pdf_colors]
        }
    }



data = get_pdf_aggregates(textual_data)

In [6]:
data['page']['size']

[Counter({72.0: 1, 28.0: 1, 4.15307092666626: 1}),
 Counter({8.0: 105, 7.0: 23, 24.0: 1}),
 Counter({7.959899425506592: 52, 8.0: 23, 16.0: 6, 14.0: 1, 24.0: 1}),
 Counter({7.0: 55,
          6.5: 8,
          9.0: 7,
          8.0: 4,
          6.634067535400391: 4,
          5.848076343536377: 3,
          6.0: 2,
          5.878775596618652: 2,
          5.247000217437744: 1,
          6.123724460601807: 1,
          6.800000190734863: 1,
          24.0: 1,
          5.6921000480651855: 1}),
 Counter({9.0: 17,
          8.0: 12,
          9.199999809265137: 10,
          6.498578071594238: 5,
          8.132010459899902: 2,
          16.0: 2,
          7.0: 2,
          7.115546226501465: 2,
          40.0: 1,
          25.086999893188477: 1,
          6.0: 1,
          6.893700122833252: 1,
          11.0: 1,
          24.0: 1}),
 Counter({7.0: 49,
          6.599999904632568: 13,
          6.5: 10,
          9.0: 8,
          6.0: 5,
          8.0: 4,
          5.878775596618652: 1

In [80]:
def get_data_from_page(data:list, page:int)->list:
    
    block_data = data['blocks']
    return block_data[page-1]

def print_page(data:list, output_path:str):
    new_doc = fitz.open(output_path)
    


# page_data = get_data_from_page(textual_data, 4)
# page_data.sort(key= lambda k: (k['bbox'][1], k['bbox'][0]))

In [ ]:
def process_page_blocks(page_data):
    
    text_color_size = []
    for block in page_data:
        bbox = block['bbox']  # Bounding box of the block
        block_text = []  # List to collect text from all spans in this block
        block_size = []
        block_color = []
        for line in block['lines']:
            for span in line['spans']:
                block_text.append(span['text'])
                block_size.append(span['size'])
                block_color.append(span['color'])
        stitched_text = '\t'.join(block_text)  # Join all text into a single string
        print(f"BBox: {bbox}, text: {stitched_text} {block_color}, {block_size}")
        text_color_size.append((stitched_text, block_color[0], block_size[0]))
        
    return text_color_size

# Example usage with your structured data
process_page_blocks(page_data)

In [ ]:
indices_variations = {
    "Additional Amount": ["Addl. Amount", "Add. Amt"],
    "Additional Multiple": ["Additional Multiples", "Addl. Multiple", "Add. Mult."],
    "Alpha": ["Alpha", "Alpha Ratio", "Alpha Coefficient"],
    "Average Asset Under Management (AAUM)": ["Average AUM", "AAUM", "Avg. AUM", "Avg Asset Under Management"],
    "Average Dividend Yield": ["Avg. Dividend Yield", "Average Div Yield"],
    "Average Maturity": ["Avg. Maturity", "Average Maturity of Bonds"],
    "Average P/B": ["Avg P/B Ratio", "Average Price to Book Ratio"],
    "Average P/E": ["Avg P/E Ratio", "Average Price to Earnings Ratio"],
    "Benchmark Index": ["Benchmark", "Index Benchmark"],
    "Benchmark Riskometer": ["Benchmark Risk Level", "Riskometer for Benchmark", "Benchmark Risk-o-meter"],
    "Creation Unit Size": ["Creation Unit Size", "Unit Size of Creation", "ETF Creation Size"],
    "Downside Deviation": ["Downside Deviation", "Downside Risk Measure", "Downside Variability"],
    "Face Value": ["Face Value", "Nominal Value", "Par Value"],
    "Fund Manager": ["Fund Manager", "Portfolio Manager", "Managed By"],
    "Inception Date": ["Inception Date", "Fund Start Date", "Date of Inception"],
    "Information Ratio": ["Information Ratio", "Info Ratio", "IR"],
    "Macaulay Duration": ["Macaulay Duration", "Duration (Macaulay)", "Macaulay's Duration"],
    "Managing Scheme": ["Managing Scheme", "Scheme Management", "Fund Scheme"],
    "Minimum Investment": ["Minimum Investment", "Min Invest", "Minimum Invest Amount"],
    "Minimum Multiple": ["Minimum Multiple", "Min Multiple", "Min. Mult"],
    "Modified Duration": ["Modified Duration", "Mod Duration", "Duration (Modified)"],
    "PB": ["PB Ratio", "Price to Book", "P/B"],
    "PE": ["PE Ratio", "Price to Earnings", "P/E"],
    "Portfolio Beta": ["Portfolio Beta", "Beta of Portfolio", "Portfolio's Beta"],
    "Portfolio Turnover Ratio": ["Portfolio Turnover", "Turnover Ratio", "Portfolio Turnover Rate"],
    "R Squared Ratio - Fund": ["R Squared", "R^2", "R-Squared for Fund"],
    "ROE": ["Return on Equity", "ROE", "Equity Returns"],
    "Scheme Name": ["Scheme Name", "Fund Name", "Investment Scheme"],
    "Scheme Objective": ["Scheme Objective", "Investment Objective", "Fund Goals"],
    "Scheme Riskometer": ["Scheme Riskometer", "Risk Level", "Investment Risk Indicator", "Scheme Risk-o-Meter"],
    "Sharpe Ratio": ["Sharpe Ratio", "Sharpe's Ratio", "Risk Adjusted Return Ratio"],
    "Sortino Ratio": ["Sortino Ratio", "Sortino's Ratio", "Downside Risk Ratio"],
    "Standard Deviation": ["Standard Deviation", "Std Dev", "Volatility Measure"],
    "Total Experience": ["Total Experience", "Years of Experience", "Management Experience"],
    "Tracking Error": ["Tracking Error", "Track Error", "Performance Deviation"],
    "Treynor Ratio": ["Treynor Ratio", "Treynor's Ratio", "Risk Adjusted Measure"],
    "Upside Deviation": ["Upside Deviation", "Upside Risk", "Positive Deviation"],
    "Yield To Maturity": ["Yield to Maturity", "YTM", "Maturity Yield"]
}

In [4]:
indices_variations = {
    "additional amount": ["additional amount", "addl. amount", "add. amt"],
    "additional multiple": ["additional multiples", "addl. multiple", "add. mult."],
    "alpha": ["alpha", "alpha ratio", "alpha coefficient"],
    "average asset under management (aaum)": ["average aum", "aaum", "avg. aum", "avg asset under management"],
    "average dividend yield": ["avg. dividend yield", "average div yield"],
    "average maturity": ["avg. maturity", "average maturity of bonds"],
    "average p/b": ["avg p/b ratio", "average price to book ratio"],
    "average p/e": ["avg p/e ratio", "average price to earnings ratio"],
    "benchmark index": ["benchmark", "index benchmark"],
    "benchmark riskometer": ["benchmark risk level", "riskometer for benchmark"],
    "creation unit size": ["creation unit size", "unit size of creation", "etf creation size"],
    "downside deviation": ["downside deviation", "downside risk measure", "downside variability"],
    "face value": ["face value", "nominal value", "par value"],
    "fund manager": ["fund manager", "portfolio manager", "managed by"],
    "inception date": ["inception date", "fund start date", "date of inception"],
    "information ratio": ["information ratio", "info ratio", "ir"],
    "macaulay duration": ["macaulay duration", "duration (macaulay)", "macaulay's duration"],
    "managing scheme": ["managing scheme", "scheme management", "fund scheme"],
    "minimum investment": ["minimum investment", "min invest", "minimum invest amount"],
    "minimum multiple": ["minimum multiple", "min multiple", "min. mult"],
    "modified duration": ["modified duration", "mod duration", "duration (modified)"],
    "pb": ["pb ratio", "price to book", "p/b"],
    "pe": ["pe ratio", "price to earnings", "p/e"],
    "portfolio beta": ["portfolio beta", "beta of portfolio", "portfolio's beta"],
    "portfolio turnover ratio": ["portfolio turnover", "turnover ratio", "portfolio turnover rate"],
    "r squared ratio - fund": ["r squared", "r^2", "r-squared for fund"],
    "roe": ["return on equity", "roe", "equity returns"],
    "scheme name": ["scheme name", "fund name", "investment scheme"],
    "scheme objective": ["scheme objective", "investment objective", "fund goals"],
    "scheme riskometer": ["scheme riskometer", "risk level", "investment risk indicator"],
    "sharpe ratio": ["sharpe ratio", "sharpe's ratio", "risk adjusted return ratio"],
    "sortino ratio": ["sortino ratio", "sortino's ratio", "downside risk ratio"],
    "standard deviation": ["standard deviation", "std dev", "volatility measure"],
    "total experience": ["total experience", "years of experience", "management experience"],
    "tracking error": ["tracking error", "track error", "performance deviation"],
    "treynor ratio": ["treynor ratio", "treynor's ratio", "risk adjusted measure"],
    "upside deviation": ["upside deviation", "upside risk", "positive deviation"],
    "yield to maturity": ["yield to maturity", "ytm", "maturity yield"]
}


In [21]:
import fitz  # PyMuPDF
import re  # Regular expressions

def check_indices_variations_and_highlight(indices_variations, path):
    doc = fitz.open(path)
    important_pages = set()

    for page_number, page in enumerate(doc):
        text_instances = page.get_text("dict")["blocks"]  # Extract text in dictionary format

        for block in text_instances:
            if "lines" in block:  # Check if lines key exists
                for line in block["lines"]:  # Navigate through lines in each block
                    for span in line["spans"]:  # Navigate through spans in each line
                        # Check text attributes
                        if span['flags'] & 4:  # Example conditions for bold or larger text
                            span_text = span['text'].lower()
                            for key, variations in indices_variations.items():
                                search_terms = [key] + variations
                                for term in search_terms:
                                    pattern = r'\b' + re.escape(term.lower()) + r'\b'
                                    if re.search(pattern, span_text):
                                        important_pages.add(page_number + 1)
                                        # Highlight found terms
                                        rect = fitz.Rect(span['bbox'])  # Get the bounding box for the span
                                        page.add_highlight_annot(rect)
                                        break  # Optional: break if only one highlight per span is needed

    if important_pages:
        output_path = path.replace('.pdf', '_highlighted.pdf')
        doc.save(output_path)
        doc.close()
        return list(important_pages), output_path
    else:
        doc.close()
        return list(important_pages), None

# Example usage
pdf_path = path + tata_path
result, modified_pdf = check_indices_variations_and_highlight(indices_variations, pdf_path)
print("Important pages:", result)
print("Modified PDF saved as:", modified_pdf)


Important pages: [1, 108, 109]
Modified PDF saved as: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\files\TataFactSheet2024_highlighted.pdf


In [22]:
from webcolors import rgb_to_name

def get_color_name(rgb_tuple):
  """
  Finds the closest named color for a given RGB value.

  Args:
    rgb_tuple: The RGB value as a tuple (r, g, b).

  Returns:
    The name of the closest color, or 'Unknown' if no close match is found.
  """
  try:
    color_name = rgb_to_name(rgb_tuple, spec='css3')  # Use CSS3 color names
    return color_name
  except ValueError:
    return 'Unknown'

# Example usage:
rgb_value = (255, 0, 0)  # Red
color_name = get_color_name(rgb_value)
print(f"Color name: {color_name}") 

rgb_value = (128, 128, 128)  # Gray
color_name = get_color_name(rgb_value)
print(f"Color name: {color_name}") 

rgb_value = (100, 50, 200)  # A custom color
color_name = get_color_name(rgb_value)
print(f"Color name: {color_name}")

Color name: red
Color name: gray
Color name: Unknown
